In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
pkmn= pd.read_csv('300k.csv')

In [3]:
pkmn[['city','latitude', 'longitude']].head(10)

,city,latitude,longitude
0,Mexico_City,20.525745,-97.460829
1,Mexico_City,20.523695,-97.461167
2,New_York,38.903590,-77.199780
3,Los_Angeles,47.665903,-122.312561
4,Los_Angeles,47.666454,-122.311628
5,Perth,-31.954980,115.853609
6,Perth,-31.954245,115.852038
7,Chicago,26.235257,-98.197591
8,Mexico_City,20.525554,-97.458800
9,New_York,32.928558,-84.340278


In [4]:
pk = pkmn.drop(['_id', 'cellId_90m', 'cellId_180m', 'cellId_370m', 'cellId_730m', 'cellId_1460m', 'cellId_2920m', 'cellId_5850m','gymIn100m', 'gymIn250m', 'gymIn500m', 'gymIn1000m', 'gymIn2500m', 'gymIn5000m', 'pokestopIn100m', 'pokestopIn250m', 'pokestopIn500m', 'pokestopIn1000m', 'pokestopIn2500m', 'pokestopIn5000m','appearedDayOfWeek'],1)

In [5]:
pk.head()

,pokemonId,latitude,longitude,appearedLocalTime,appearedTimeOfDay,appearedHour,appearedMinute,appearedDay,appearedMonth,appearedYear,...,cooc_143,cooc_144,cooc_145,cooc_146,cooc_147,cooc_148,cooc_149,cooc_150,cooc_151,class
0,16,20.525745,-97.460829,2016-09-08T03:57:45,night,5,57,8,8,2016,...,False,False,False,False,False,False,False,False,False,16
1,133,20.523695,-97.461167,2016-09-08T03:57:37,night,5,57,8,8,2016,...,False,False,False,False,False,False,False,False,False,133
2,16,38.903590,-77.199780,2016-09-08T03:57:25,night,5,57,8,8,2016,...,False,False,False,False,False,False,False,False,False,16
3,13,47.665903,-122.312561,2016-09-08T03:56:22,night,5,56,8,8,2016,...,False,False,False,False,False,False,False,False,False,13
4,133,47.666454,-122.311628,2016-09-08T03:56:08,night,5,56,8,8,2016,...,False,False,False,False,False,False,False,False,False,133


In [6]:
#solve the time problem
pk['appearedLocalTime'] =  pd.to_datetime(pk['appearedLocalTime'], format='%Y-%m-%dT%H:%M:%S') 
pk = pk.drop(['appearedHour', 'appearedMinute', 'appearedDay', 'appearedMonth', 'appearedYear'],1)
pk['appearedHour'] = pk['appearedLocalTime'].dt.hour
pk['appearedMinute'] = pk['appearedLocalTime'].dt.minute
pk['appearedDay'] = pk['appearedLocalTime'].dt.day
pk['appearedMonth'] = pk['appearedLocalTime'].dt.month
pk['appearedYear'] = pk['appearedLocalTime'].dt.year
pk = pk.drop(['appearedLocalTime'],1)

In [7]:
# convert time to code
time = {"morning": 1, "afternoon": 2, "evening": 3, "night": 4}
pk['appearedTimeOfDay'] = pk['appearedTimeOfDay'].map(time)

In [8]:
#redefining continents 
pk.loc[pk['continent']=='America/Indiana','continent']='America'
pk.loc[pk['continent']=='America/Kentucky','continent']='America'
pk.loc[pk['continent']=='Pacific','continent']='Australia'
pk.loc[pk['continent']=='Atlantic','continent']='Europe'
pk.loc[pk['continent']=='America/Argentina','continent']='America'
pk.loc[pk['continent']=='India','continent']='Asia'

In [9]:
cont = {'America':1, 'Australia': 2, 'Europe': 3, 'Asia': 4}
pk['continent'] = pk['continent'].map(cont)

In [10]:
pk.head(200)

,pokemonId,latitude,longitude,appearedTimeOfDay,terrainType,closeToWater,city,continent,weather,temperature,...,cooc_148,cooc_149,cooc_150,cooc_151,class,appearedHour,appearedMinute,appearedDay,appearedMonth,appearedYear
0,16,20.525745,-97.460829,4,14,False,Mexico_City,1.0,Foggy,25.5,...,False,False,False,False,16,3,57,8,9,2016
1,133,20.523695,-97.461167,4,14,False,Mexico_City,1.0,Foggy,25.5,...,False,False,False,False,133,3,57,8,9,2016
2,16,38.903590,-77.199780,4,13,False,New_York,1.0,Clear,24.2,...,False,False,False,False,16,3,57,8,9,2016
3,13,47.665903,-122.312561,4,0,True,Los_Angeles,1.0,PartlyCloudy,15.6,...,False,False,False,False,13,3,56,8,9,2016
4,133,47.666454,-122.311628,4,0,True,Los_Angeles,1.0,PartlyCloudy,15.6,...,False,False,False,False,133,3,56,8,9,2016
5,21,-31.954980,115.853609,4,13,False,Perth,2.0,PartlyCloudy,16.5,...,False,False,False,False,21,3,55,8,9,2016
6,66,-31.954245,115.852038,4,13,False,Perth,2.0,PartlyCloudy,16.5,...,False,False,False,False,66,3,55,8,9,2016
7,27,26.235257,-98.197591,4,13,False,Chicago,1.0,Clear,28.0,...,False,False,False,False,27,3,55,8,9,2016
8,35,20.525554,-97.458800,4,14,False,Mexico_City,1.0,Foggy,25.5,...,False,False,False,False,35,3,55,8,9,2016
9,19,32.928558,-84.340278,4,8,False,New_York,1.0,Clear,23.7,...,False,False,False,False,19,3,54,8,9,2016


In [11]:
pk['windBearing'].head()

0    269
1    269
2    218
3    160
4    160
Name: windBearing, dtype: int64

In [12]:
#encode the windBearing
pk.loc[(pk['windBearing'] >= 400), 'windBearing'] = 0
pk.loc[(pk['windBearing'] < 24), 'windBearing'] = 0
pk.loc[(pk['windBearing'] >= 24) & (pk['windBearing'] <70), 'windBearing'] = 1
pk.loc[(pk['windBearing'] >= 70) & (pk['windBearing'] < 120), 'windBearing'] = 2
pk.loc[(pk['windBearing'] >= 120) & (pk['windBearing'] < 170), 'windBearing'] = 3
pk.loc[(pk['windBearing'] >= 170) & (pk['windBearing'] < 220), 'windBearing'] = 4
pk.loc[(pk['windBearing'] >= 220) & (pk['windBearing'] < 270), 'windBearing'] = 5
pk.loc[(pk['windBearing'] >= 270) & (pk['windBearing'] <320), 'windBearing'] = 6
pk.loc[(pk['windBearing'] >= 320) & (pk['windBearing'] < 400), 'windBearing'] = 7

In [13]:
#encode the weather
weather = {'Foggy': 1, 'Clear': 2, 'PartlyCloudy': 3, 'MostlyCloudy': 4,"Overcast" : 5,"Rain" : 6,"BreezyandOvercast" : 7,"LightRain" : 8, "Drizzle" : 9,"BreezyandPartlyCloudy" : 10,"HeavyRain" : 11,"BreezyandMostlyCloudy" :12,
          "Breezy" : 13, "Windy": 14, "WindyandFoggy" : 15, "Humid" : 16, "Dry" : 17, "WindyandPartlyCloudy" : 18, "DangerouslyWindy":19, "DryandMostlyCloudy": 20, "DryandPartlyCloudy" : 21, "DrizzleandBreezy":22,"LightRainandBreezy" :23, "HumidandPartlyCloudy" :24, "HumidandOvercast" : 25,"RainandWindy" : 26}

In [14]:
pk['weather'] = pk['weather'].map(weather)
pk.head()

,pokemonId,latitude,longitude,appearedTimeOfDay,terrainType,closeToWater,city,continent,weather,temperature,...,cooc_148,cooc_149,cooc_150,cooc_151,class,appearedHour,appearedMinute,appearedDay,appearedMonth,appearedYear
0,16,20.525745,-97.460829,4,14,False,Mexico_City,1.0,1,25.5,...,False,False,False,False,16,3,57,8,9,2016
1,133,20.523695,-97.461167,4,14,False,Mexico_City,1.0,1,25.5,...,False,False,False,False,133,3,57,8,9,2016
2,16,38.903590,-77.199780,4,13,False,New_York,1.0,2,24.2,...,False,False,False,False,16,3,57,8,9,2016
3,13,47.665903,-122.312561,4,0,True,Los_Angeles,1.0,3,15.6,...,False,False,False,False,13,3,56,8,9,2016
4,133,47.666454,-122.311628,4,0,True,Los_Angeles,1.0,3,15.6,...,False,False,False,False,133,3,56,8,9,2016


In [15]:
#encode the population_density
pk.loc[pk['population_density'] < 200, 'population_density'] = 0
pk.loc[(pk['population_density'] >= 200) & (pk['population_density'] < 800), 'population_density'] = 1
pk.loc[(pk['population_density'] >= 800) & (pk['population_density'] < 1400), 'population_density'] = 2
pk.loc[(pk['population_density'] > 1400), 'population_density'] = 3
pk = pk.drop(['urban', 'suburban', 'midurban', 'rural'],1)

In [16]:
pk['population_density'].head()

0    3.0
1    3.0
2    1.0
3    3.0
4    3.0
Name: population_density, dtype: float64

In [17]:
# encode the temperature
pk.loc[pk['temperature'] < 10, 'temperature'] = 0
pk.loc[(pk['temperature'] >= 10) & (pk['temperature'] < 20), 'temperature'] = 1
pk.loc[(pk['temperature'] >= 20) & (pk['temperature'] < 30), 'temperature'] = 2
pk.loc[(pk['temperature'] >= 30)& (pk['temperature'] < 40), 'temperature'] = 3
pk.loc[(pk['temperature'] > 40), 'temperature'] = 4

In [18]:
pk['temperature'].head()

0    2.0
1    2.0
2    2.0
3    1.0
4    1.0
Name: temperature, dtype: float64

In [19]:
#encode the windspeed
pk.loc[pk['windSpeed'] < 1, 'windSpeed'] = 0
pk.loc[(pk['windSpeed'] >= 1) & (pk['windSpeed'] < 5), 'windSpeed'] = 1
pk.loc[(pk['windSpeed'] >= 5) & (pk['windSpeed'] < 10), 'windSpeed'] = 2
pk.loc[(pk['windSpeed'] >= 10)& (pk['windSpeed'] < 15), 'windSpeed'] = 3
pk.loc[(pk['windSpeed'] >= 15)& (pk['windSpeed'] < 20), 'windSpeed'] = 4
pk.loc[(pk['windSpeed'] >= 20)& (pk['windSpeed'] < 25), 'windSpeed'] = 5
pk.loc[(pk['windSpeed'] >= 25)& (pk['windSpeed'] < 30), 'windSpeed'] = 6
pk.loc[(pk['windSpeed'] >= 30)& (pk['windSpeed'] < 35), 'windSpeed'] = 7
pk.loc[(pk['windSpeed'] > 35), 'windSpeed'] = 8

In [20]:
pk['windSpeed'].head()

0    1.0
1    1.0
2    1.0
3    2.0
4    2.0
Name: windSpeed, dtype: float64

In [21]:
#encode the pressure
pk.loc[pk['pressure'] < 1000, 'pressure'] = 0
pk.loc[(pk['pressure'] >= 1000) & (pk['pressure'] < 1005), 'pressure'] = 1
pk.loc[(pk['pressure'] >= 1005) & (pk['pressure'] < 1010), 'pressure'] = 2
pk.loc[(pk['pressure'] >= 1010)& (pk['pressure'] < 1015), 'pressure'] = 3
pk.loc[(pk['pressure'] >= 1015)& (pk['pressure'] < 1020), 'pressure'] = 4
pk.loc[(pk['pressure'] >= 1020)& (pk['pressure'] < 1025), 'pressure'] = 5
pk.loc[(pk['pressure'] >= 1025)& (pk['pressure'] < 1030), 'pressure'] = 6
pk.loc[(pk['pressure'] > 1030), 'pressure'] = 7

In [22]:
#whether close to water?
pk['closeToWater'] = pk['closeToWater'].astype('category')
pk['closeToWater'] = pk['closeToWater'].cat.codes

In [23]:
select = ['class','latitude', 'longitude','city','appearedHour','appearedMinute','appearedTimeOfDay','terrainType','pressure','windBearing','weather','population_density','temperature','windSpeed','closeToWater']

In [24]:
data = pk.loc[:,select]

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
features = data.drop(['latitude','longitude','city'],1)
labels = pd.concat([data['latitude'], data['longitude']],axis = 1)

In [28]:
train_feature, test_feature,train_label, test_label = train_test_split(features, labels, train_size = 0.7,random_state = 46)

In [31]:
from sklearn import ensemble
from sklearn.externals import joblib

In [32]:
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(max_depth=10,n_estimators=100,learning_rate=0.7,verbose=1)
model = model_GradientBoostingRegressor
model.fit(train_feature,train_label.iloc[:,[0]])
joblib.dump(model, '../final project/model_latitude.pkl')
score = model.score(test_feature,test_label.iloc[:,[0]])
result = model.predict(test_feature)
print(score)

      Iter       Train Loss   Remaining Time 
         1         111.2774            6.01m
         2          75.6403            5.85m
         3          53.9645            6.28m
         4          49.4695            6.03m
         5          41.9584            5.92m
         6          32.1886            6.01m
         7          30.7217            5.71m
         8          29.1536            5.74m
         9          26.3956            5.79m
        10          25.0641            5.75m
        20          14.1599            5.06m
        30           9.3750            4.41m
        40           7.4270            3.67m
        50           6.0016            3.09m
        60           4.8693            2.47m
        70           4.0691            1.85m
        80           3.4321            1.24m
        90           3.0412           36.95s
       100           2.6685            0.00s
0.9425131819012504


In [33]:
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(max_depth=10,n_estimators=100,learning_rate=0.7,verbose=1)
model2 = model_GradientBoostingRegressor
model2.fit(train_feature,train_label.iloc[:,[1]])
joblib.dump(model2, '../final project/model_longitude.pkl')


      Iter       Train Loss   Remaining Time 
         1        1696.9952            6.28m
         2        1082.1340            6.53m
         3         806.2381            6.63m
         4         693.3527            6.49m
         5         612.3431            6.19m
         6         508.8254            6.17m
         7         471.8031            5.93m
         8         417.3699            5.87m
         9         402.1630            5.87m
        10         372.6903            5.74m
        20         214.5787            4.83m
        30         131.0001            4.36m
        40         101.7457            3.78m
        50          83.2694            3.17m
        60          71.5559            2.52m
        70          59.4545            1.90m
        80          51.8432            1.27m
        90          43.2903           38.40s
       100          37.5246            0.00s
-1.3403045991438565


In [34]:
score = model2.score(test_feature,test_label.iloc[:,[1]])
result_longitude = model2.predict(test_feature)
print(score)

0.9585314863841073


In [35]:
latitude_result = pd.DataFrame(result)
longitude_result = pd.DataFrame(result_longitude)
latitude_result.to_csv('../final project/latitude_result.csv')
longitude_result.to_csv('../final project/longitude_result.csv')

In [41]:
print(result)

[49.45595098 46.7403312  52.1849592  ... 55.92653483 56.23249589
 50.47918253]


In [42]:
print(result_longitude)

[  17.27204126 -122.53420679    8.65899685 ...   23.0354363    15.3413423
   14.87146513]


In [43]:
test_label.head()

,latitude,longitude
208428,49.962754,14.077851
161584,45.514116,-122.883598
243431,49.847894,18.168771
137138,49.644354,-1.605360
43466,32.905209,-84.328608


In [44]:
model_latitude = joblib.load('../final project/model_latitude.pkl')
model_longitude = joblib.load('../final project/model_longitude.pkl')

In [45]:
test_feature.head()

,class,appearedHour,appearedMinute,appearedTimeOfDay,terrainType,pressure,windBearing,weather,population_density,temperature,windSpeed,closeToWater
208428,118,11,9,2,5,3.0,5,4,1.0,2.0,4.0,0
161584,19,5,15,1,13,4.0,7,2,2.0,1.0,2.0,0
243431,16,19,2,4,13,4.0,4,3,2.0,1.0,1.0,0
137138,96,12,45,2,12,5.0,5,5,3.0,1.0,3.0,1
43466,10,21,26,4,8,5.0,3,17,0.0,3.0,1.0,0


In [56]:
pclass = 118
hour = 11
minute = 9
day = 2
terrainType = 5
pressure = 3.0
windBearing = 5
weather = 4
population_density = 1.0
temperature = 2.0
windSpeed = 4.0
closeToWater = 0
from collections import OrderedDict
feature_list = ['class','appearedHour','appearedMinute','appearedTimeOfDay','terrainType','pressure',
                'windBearing','weather','population_density','temperature','windSpeed','closeToWater']
d = {'class':pclass,'appearedHour':hour,'appearedMinute':minute,'appearedTimeOfDay':day,'terrainType':terrainType,
    'pressure':pressure,'windBearing':windBearing,'weather':weather,'population_density':population_density,
    'temperature':temperature,'windSpeed':windSpeed,'closeToWater':closeToWater}
row = d
d2 = OrderedDict()
for key in feature_list:
    if row.get(key) is not None:
        d2[key] = row.get(key)
feature = pd.DataFrame([d2])
feature.head()

,class,appearedHour,appearedMinute,appearedTimeOfDay,terrainType,pressure,windBearing,weather,population_density,temperature,windSpeed,closeToWater
0,118,11,9,2,5,3.0,5,4,1.0,2.0,4.0,0


In [57]:
pred_latitude = model_latitude.predict(feature)
pred_longitude = model_longitude.predict(feature)

In [58]:
print(pred_latitude)

[49.45595098]


In [59]:
print(pred_longitude)

[17.27204126]


In [64]:
result = [pred_latitude[0],pred_longitude[0]]

In [65]:
print(result)

[49.45595098042733, 17.272041260717515]


In [84]:
def convert_tp(a):
    d = {'Water':0,'Evergreen Needleleaf forest':1,'Evergreen Broadleaf forest':2,'Deciduous Needleleaf forest':3,
        'Deciduous Broadleaf forest':4,'Mixed forest':5,'Closed shrublands':6,'Open shrublands':7,'Woody savannas':8,
        'Savannas':9,'Grasslands':10,'Permanent wetlands':11,'Croplands':12,'Urban and built-up':13,
         'Cropland/Natural vegetation mosaic':14,'Snow and ice':15,'Barren or sparsely vegetated':16,'Unclassified':254,
        'Fill Value':255}
    return d[a]

def convert_pr(a):
    if a < 1000:
        b = 0
    if 1000 <= a < 1005:
        b = 1
    if 1005 <= a < 1010:
        b = 2
    if 1010 <= a < 1015:
        b = 3
    if 1015 <= a <1020:
        b = 4
    if 1020 <= a < 1025:
        b = 5
    if 1025 <= a < 1030:
        b = 6
    if a >= 1030:
        b = 7
    return b

def convert_wb(a):
    if a >400:
        b = 0
    if a < 24:
        b = 0
    if 24 <= a < 70:
        b = 1
    if 70 <= a < 120:
        b = 2
    if 120 <= a < 170:
        b = 3
    if 170 <= a <220:
        b = 4
    if 220 <= a <270:
        b = 5
    if 270 <= a < 320:
        b = 6
    if 320 <= a < 400:
        b = 7
    return(b)

def convert_we(a):
    weather = {'Foggy': 1, 'Clear': 2, 'PartlyCloudy': 3, 'MostlyCloudy': 4,"Overcast" : 5,"Rain" : 6,"BreezyandOvercast" : 7,"LightRain" : 8, "Drizzle" : 9,"BreezyandPartlyCloudy" : 10,"HeavyRain" : 11,"BreezyandMostlyCloudy" :12,
          "Breezy" : 13, "Windy": 14, "WindyandFoggy" : 15, "Humid" : 16, "Dry" : 17, "WindyandPartlyCloudy" : 18, "DangerouslyWindy":19, "DryandMostlyCloudy": 20, "DryandPartlyCloudy" : 21, "DrizzleandBreezy":22,"LightRainandBreezy" :23,
               "HumidandPartlyCloudy" :24, "HumidandOvercast" : 25,"RainandWindy" : 26}
    return weather[a]

def convert_pd(a):
    if a < 200:
        b = 0
    if 200 <= a < 800:
        b = 1
    if 800 <= a < 1400:
        b = 2
    if a >= 1400:
        b = 3
    return(b)

def convert_tm(a):
    if a <10:
        b = 0
    if 10 <= a < 20:
        b = 1
    if 20 <= a < 30:
        b = 2
    if 30 <= a < 40:
        b = 3
    if a >= 40:
        b = 4
    return b

def convert_ws(a):
    if a < 1:
        b = 0
    if 1 <= a < 5:
        b = 1
    if 5 <= a < 10:
        b = 2
    if 10 <= a < 15:
        b = 3
    if 15 <= a < 20:
        b = 4
    if 20 <= a < 25:
        b = 5
    if 25 <= a < 30:
        b = 6
    if 30 <= a < 35:
        b = 7
    if a >= 35:
        b = 8
    return b

def convert_water(a):
    d = {'True':0,'False':1}
    return d[a]
        

In [86]:
print(convert_ws(13.5))

3
